In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Resize, Compose
from torchvision.datasets import Cityscapes
from torch.utils.data import random_split
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import utils
import wandb



In [2]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: svenbbs. Use `wandb login --relogin` to force relogin


True

In [3]:
lr_rate = 1e-3
epochs = 10
batch_size = 64
subsize = (128, 256)
momentum = 0

In [4]:

transform = Compose([
    Resize(subsize),
    ToTensor()
])


target_transform = Compose([
    Resize(subsize),
    ToTensor()
])

#Desktop
dataset = Cityscapes(root="E:\CityScapes", split='train', mode='fine', target_type='semantic', transform=transform, target_transform=target_transform)

#Laptop
#dataset = Cityscapes(root="C:/Users/20182573/Documents/CityScapes", split='train', mode='fine', target_type='semantic', transform=transform, target_transform=target_transform)

#subset_small, subset_big = random_split(dataset, [0.2,0.8])
train_dataset, val_dataset = random_split(dataset, [0.8,0.2])

In [5]:
len(train_dataset)

2380

In [6]:


train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=8)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [7]:

from model import Model

model = Model().cuda()
loss_fn = nn.CrossEntropyLoss(ignore_index=255)
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate, momentum=momentum)


In [8]:
def train(dataloader, model, loss_fn, optimizer, run):
    """
    Train a model for 1 epoch.

    Params:
    - dataloader:   dataset to train on.
    - model:        the model object to be trained.
    - loss_fn:      the loss function.
    - optimizer:    the desired optimization.
    """
    size = len(dataloader.dataset)
    model.train() #Set the model to train mode
    for batch, (IMG,SEGM) in enumerate(dataloader):
        IMG = IMG.to('cuda')
        SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
        SEGM = utils.map_id_to_train_id(SEGM).to('cuda')
        
        #predict
        pred = model(IMG)
        #Loss
        loss = loss_fn(pred, SEGM)
        

        #Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #print loss during training
        loss, current = loss.item(), (batch + 1) * len(IMG)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        run.log({"train_loss": loss})
    
    return run
            


In [9]:
def test(dataloader, model, loss_fn):
    """
    Test a model.

    Params:
    - dataloader:   dataset to test on.
    - model:        the model object to be tested.
    - loss_fn:      the loss function.
    """
    num_batches = len(dataloader)
    model.eval() #model in eval mode
    test_loss = 0
    with torch.no_grad():
        for _, (IMG,SEGM) in enumerate(dataloader):
            IMG = IMG.to('cuda')
            SEGM  = (SEGM*255).long().squeeze()     #*255 because the id are normalized between 0-1
            SEGM = utils.map_id_to_train_id(SEGM).to('cuda')

            pred = model(IMG)
            test_loss += loss_fn(pred, SEGM).item()
            
    test_loss /= num_batches
    print(f"Test Error: \n Avg loss: {test_loss:>8f} \n")
    return test_loss
    


In [10]:



run = wandb.init(
    # Set the project where this run will be logged
    project="CS_challenge", name="Baseline",
    # Track hyperparameters and run metadata
    config={
        "learning_rate": {lr_rate},
        "epochs": {epochs},
        "Momentum": {momentum},
        "Batch_size": {batch_size},
        "model version": 0.0,
        "subset size [%]": 100,
        "resize": {subsize},
    },
)




for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    run = train(train_dataloader, model, loss_fn, optimizer, run)
    test_loss = test(val_dataloader, model, loss_fn)
    run.log({"Test_loss": test_loss})
print("Done!")

run.finish()

Epoch 1
-------------------------------
loss: 3.680243  [   64/ 2380]
loss: 3.690689  [  128/ 2380]
loss: 3.662894  [  192/ 2380]
loss: 3.661673  [  256/ 2380]
loss: 3.651207  [  320/ 2380]
loss: 3.632881  [  384/ 2380]
loss: 3.632411  [  448/ 2380]
loss: 3.628073  [  512/ 2380]
loss: 3.615046  [  576/ 2380]
loss: 3.591819  [  640/ 2380]
loss: 3.594000  [  704/ 2380]
loss: 3.571939  [  768/ 2380]
loss: 3.566234  [  832/ 2380]
loss: 3.557127  [  896/ 2380]
loss: 3.545800  [  960/ 2380]
loss: 3.540694  [ 1024/ 2380]
loss: 3.525391  [ 1088/ 2380]
loss: 3.523522  [ 1152/ 2380]
loss: 3.508590  [ 1216/ 2380]
loss: 3.496364  [ 1280/ 2380]
loss: 3.484048  [ 1344/ 2380]
loss: 3.492723  [ 1408/ 2380]
loss: 3.474499  [ 1472/ 2380]
loss: 3.463879  [ 1536/ 2380]
loss: 3.444469  [ 1600/ 2380]
loss: 3.467240  [ 1664/ 2380]
loss: 3.455887  [ 1728/ 2380]
loss: 3.433334  [ 1792/ 2380]
loss: 3.444580  [ 1856/ 2380]
loss: 3.408376  [ 1920/ 2380]
loss: 3.418385  [ 1984/ 2380]
loss: 3.394811  [ 2048/ 2380]


Test_loss,█▇▆▅▄▃▂▂▁▁
train_loss,██▇▇▇▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁
Test_loss,2.13868
train_loss,2.19297


In [11]:
torch.save(model.state_dict(), "model.pth")

In [12]:
wandb.finish()